# Preprocessing

In [36]:
import pandas as pd 
import numpy as np

In [37]:
basics_url = "https://datasets.imdbws.com/title.basics.tsv.gz"

In [38]:
ratings_url = "https://datasets.imdbws.com/title.ratings.tsv.gz"

In [39]:
akas_url = "https://datasets.imdbws.com/title.akas.tsv.gz"

In [40]:
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)

In [41]:
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

In [42]:
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

In [43]:
basics.head(3)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"


In [44]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9306100 entries, 0 to 9306099
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 639.0+ MB


In [45]:
basics.describe()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
count,9306100,9306100,9306089,9306089,9306100,9306100,9306100,9306100,9306090
unique,9306100,11,4263436,4284081,9,151,98,877,2331
top,tt0000001,tvEpisode,Episode #1.1,Episode #1.1,0,\N,\N,\N,Drama
freq,1,7021695,45506,45506,9017249,1244688,9208579,6794754,1056962


In [46]:
basics.replace({'\\N':np.nan}, inplace = True)

In [47]:
basics = basics.dropna(subset=['runtimeMinutes', 'genres'])

In [48]:
# remove all but 'movie'
filter = basics['titleType'] == 'movie'
basics= basics.loc[filter]

In [49]:
# Exclude movies that are included in the documentary category.
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

In [50]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 281625 entries, 8 to 9305949
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          281625 non-null  object
 1   titleType       281625 non-null  object
 2   primaryTitle    281625 non-null  object
 3   originalTitle   281625 non-null  object
 4   isAdult         281625 non-null  object
 5   startYear       277248 non-null  object
 6   endYear         0 non-null       object
 7   runtimeMinutes  281625 non-null  object
 8   genres          281625 non-null  object
dtypes: object(9)
memory usage: 21.5+ MB


In [51]:
basics['startYear'] = basics['startYear'].astype(float)

In [52]:
# only keeping 2000-2002
filter2 = (basics['startYear'] >= 2000) & (basics['startYear'] <= 2022)
basics = basics[filter2]

In [53]:
akas.replace({'\\N':np.nan}, inplace = True)

In [54]:
akas.dropna(subset=['region'], inplace=True)

In [55]:
akas.isna().sum()

titleId                   0
ordering                  0
title                     3
region                    0
language            4446131
types              28164787
attributes         31440161
isOriginalTitle        2175
dtype: int64

In [56]:
only_US = akas['region'].str.contains('US',case=False)
akas = akas[only_US]

In [57]:
ratings.isna().sum()

tconst           0
averageRating    0
numVotes         0
dtype: int64

In [58]:
ratings.replace({'\\N':np.nan},inplace = True)

In [59]:
keepers_ratings =ratings['tconst'].isin(akas['region'])
keepers_ratings

0          False
1          False
2          False
3          False
4          False
           ...  
1238555    False
1238556    False
1238557    False
1238558    False
1238559    False
Name: tconst, Length: 1238560, dtype: bool

In [60]:
ratings = ratings[keepers_ratings]
ratings

,tconst,averageRating,numVotes


In [61]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics['tconst'].isin(akas['region'])
keepers

34792      False
61094      False
67640      False
77934      False
86770      False
           ...  
9305772    False
9305781    False
9305820    False
9305865    False
9305949    False
Name: tconst, Length: 143970, dtype: bool

In [62]:
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres


In [63]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          0 non-null      object 
 1   titleType       0 non-null      object 
 2   primaryTitle    0 non-null      object 
 3   originalTitle   0 non-null      object 
 4   isAdult         0 non-null      object 
 5   startYear       0 non-null      float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  0 non-null      object 
 8   genres          0 non-null      object 
dtypes: float64(1), object(8)
memory usage: 0.0+ bytes


In [64]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1361344 entries, 5 to 33544157
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1361344 non-null  object
 1   ordering         1361344 non-null  int64 
 2   title            1361344 non-null  object
 3   region           1361344 non-null  object
 4   language         3695 non-null     object
 5   types            965071 non-null   object
 6   attributes       45205 non-null    object
 7   isOriginalTitle  1359969 non-null  object
dtypes: int64(1), object(7)
memory usage: 93.5+ MB


In [65]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         0 non-null      object 
 1   averageRating  0 non-null      float64
 2   numVotes       0 non-null      int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 0.0+ bytes


## Saving Data

In [66]:
import os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

['.ipynb_checkpoints',
 'title.akas.tsv.gz',
 'title.basics.tsv.gz',
 'title.ratings.tsv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz']

In [67]:
# Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [68]:
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)

In [69]:
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

In [70]:
# Open saved file and preview again
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres


In [71]:
# Open saved file and preview again
akas = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [72]:
# Open saved file and preview again
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)
ratings.head()

,tconst,averageRating,numVotes
